In [1]:
!export CUBLAS_WORKSPACE_CONFIG=:4096:8

In [1]:
import warnings
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '2'
# import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8' 

In [2]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-51yydkbl
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-51yydkbl
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


In [3]:
import torch
import clip
# !pip install git+https://github.com/openai/CLIP.git
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
device

'cuda'

In [5]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100
import torch
import math
from torch import nn
import pytorch_lightning as pl
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score,precision_score
from torch.utils.data import DataLoader, random_split
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.nn import functional as F
from torchvision.datasets import MNIST
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from PIL import Image
from torchvision import transforms
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [6]:
data = pd.read_csv('/DATA/atul_2221cs20/jitendra/ethics/cm_train.csv')  #Drive link where your data is saved
#data_test = pd.read_csv('/home/gitanjali_2021cs03/work_on_bias_final/gender_biasness/MAMI_test_scene_graph.csv')

In [7]:
data.head(5)

,label,input
0,1,I went to the principal's office to change my ...
1,0,I secured the loan because I would make the pa...
2,0,"After asking my daughter first, I discuss my d..."
3,1,I discuss my daughter's health problems with o...
4,1,"As my grandparents slept, I moved their laundr..."


In [8]:
len(data)

13910

In [34]:
from multilingual_clip import pt_multilingual_clip
import transformers

In [35]:
#just to check how multilingual clip works
texts = [
    'Three blind horses listening to Mozart.',
    'Älgen är skogens konung!',
    'Wie leben Eisbären in der Antarktis?',
    'Вы знали, что все белые медведи левши?'
]
model_name = 'M-CLIP/XLM-Roberta-Large-Vit-L-14'
# Load Model & Tokenizer
model_t = pt_multilingual_clip.MultilingualCLIP.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
embeddings = model_t.forward(texts, tokenizer)
print(embeddings.shape)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([4, 768])


In [36]:
import os
#os.environ['CUDA_VISIBLE_DEVICES']='2'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# clip_model, compose = clip.load('RN50x4', device = device)
clip_model, compose = clip.load("ViT-B/32", device = device)
# text_inputs = (clip.tokenize(data.Text_Transcription.values[321],truncate=True)).to(device)
# print(text_inputs)

In [10]:
def get_data(data):
  #data = pd.read_csv(dataset_path)
  text = list(data['input'])
#   img_path = list(data['name'])
#   name = list(data['name'])
  # label = list(data['Level1'])
  label = list(data['label'])
  text_features,image_features,l,Name,v = [],[],[],[],[]
  for txt,L in tqdm(zip(text,label)):
    # try:
    #   img = Image.open('/home/gitanjali_2021cs03/work_on_bias_final/gender_biasness/MAMI_2022_images/training_images/'+img)
    # except Exception as e:
    #   print(e)
    #   continue

    # img = torch.stack([compose(img).to(device)])
    l.append(int(L))
    # Name.append(n)
    with torch.no_grad():
        txt=txt[:77]
        #print(txt)
        temp_txt = model_t.forward(txt, tokenizer).detach().cpu().numpy()
            #print(temp_txt.shape)
#             temp_txt = model.forward(txt, tokenizer).detach().cpu().numpy()
            
        text_features.append(temp_txt)
        #temp_txt=model_t.forward(txt, tokenizer).detach().cpu().numpy()
        #text_features.append(temp_txt)
        del temp_txt
        torch.cuda.empty_cache()
        
      #temp_img = clip_model.encode_image(img).detach().cpu().numpy()
      #image_features.append(temp_img)
    
    #   del temp_img
    #del img
    #torch.cuda.empty_cache()

  return text_features,l



In [11]:
class HatefulDataset(Dataset):

  def __init__(self,data):
    self.t_f,self.label = get_data(data)
    self.t_f = np.squeeze(np.asarray(self.t_f),axis=1)
    # self.i_f = np.squeeze(np.asarray(self.i_f),axis=1)
  def __len__(self):
    return len(self.label)

  def __getitem__(self,idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    #print(idx)
    # name=self.name[idx]
    label = self.label[idx]
    T = self.t_f[idx,:]
    # I = self.i_f[idx,:]
    sample = {'label':label,'processed_txt':T}
    return sample

In [130]:
#sample_dataset = HatefulDataset(data)

0it [00:00, ?it/s]

In [133]:
sample_dataset[4]

{'label': 1,
 'processed_txt': array([-1.52348652e-01, -7.54387453e-02, -1.65186912e-01,  2.63631344e-01,
         2.20908076e-01, -1.77685574e-01, -4.23201114e-01,  1.34160891e-01,
         9.94548053e-02, -3.46936695e-02,  1.57933071e-01, -2.17930786e-02,
         3.13578576e-01,  1.24466382e-02, -3.04684609e-01,  3.53767842e-01,
         3.65526080e-01, -1.23490877e-01,  3.28399474e-03, -1.14590302e-02,
        -3.43225390e-01,  1.87817276e-01, -6.19067019e-03, -1.21930800e-01,
         5.32314658e-01, -2.31333449e-01, -1.09672010e-01,  3.25455852e-02,
         4.23649043e-01, -1.44125357e-01, -2.86487490e-01, -1.98219806e-01,
        -1.82634786e-01, -5.94884455e-02,  7.19535530e-01, -3.58192384e-01,
         3.26418817e-01, -1.30252019e-01,  2.49947365e-02, -1.75835803e-01,
         1.68119103e-01, -2.25986481e-01, -1.64066255e-01,  2.79580019e-02,
        -2.69701838e-01, -2.12918147e-01, -3.39793414e-01, -1.27989044e-02,
        -6.29800022e-01,  2.44532805e-03,  5.25802597e-02,

In [134]:
#torch.save(sample_dataset,"cm_train.pt")

In [12]:
sample_dataset_new= torch.load("cm_train.pt")

In [13]:
len(sample_dataset_new)

13910

In [14]:
sample_dataset_new[4]

{'label': 1,
 'processed_txt': array([-1.52348652e-01, -7.54387453e-02, -1.65186912e-01,  2.63631344e-01,
         2.20908076e-01, -1.77685574e-01, -4.23201114e-01,  1.34160891e-01,
         9.94548053e-02, -3.46936695e-02,  1.57933071e-01, -2.17930786e-02,
         3.13578576e-01,  1.24466382e-02, -3.04684609e-01,  3.53767842e-01,
         3.65526080e-01, -1.23490877e-01,  3.28399474e-03, -1.14590302e-02,
        -3.43225390e-01,  1.87817276e-01, -6.19067019e-03, -1.21930800e-01,
         5.32314658e-01, -2.31333449e-01, -1.09672010e-01,  3.25455852e-02,
         4.23649043e-01, -1.44125357e-01, -2.86487490e-01, -1.98219806e-01,
        -1.82634786e-01, -5.94884455e-02,  7.19535530e-01, -3.58192384e-01,
         3.26418817e-01, -1.30252019e-01,  2.49947365e-02, -1.75835803e-01,
         1.68119103e-01, -2.25986481e-01, -1.64066255e-01,  2.79580019e-02,
        -2.69701838e-01, -2.12918147e-01, -3.39793414e-01, -1.27989044e-02,
        -6.29800022e-01,  2.44532805e-03,  5.25802597e-02,

In [15]:
# torch.manual_seed(123)
# t_p,v_p = torch.utils.data.random_split(sample_dataset_new,[73897,8212])

# # torch.manual_seed(123)
# t_p,te_p = torch.utils.data.random_split(t_p,[59197,14700])

In [16]:
torch.manual_seed(123)
t_p,v_p = torch.utils.data.random_split(sample_dataset_new,[13000,910])

# torch.manual_seed(123)
t_p,te_p = torch.utils.data.random_split(t_p,[10500,2500])

In [17]:
from distutils.version import LooseVersion, StrictVersion

In [18]:

class Classifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        # Initialize text and image processors
        self.text_processor = torch.nn.Linear(768, 256)  # Processed to 256 features
        self.image_processor = torch.nn.Linear(512, 256)  # Processed to 256 features

        # For concatenated features: since you concatenate text_out and image_out, the input here should be 512
        self.combined_processor = torch.nn.Linear(512, 2)  # Correct dimension for concatenated case

        # If only text features are used and no concatenation happens, you might want to consider another layer
        # to handle the case when image_features is None:
        self.text_only_processor = torch.nn.Linear(256, 2)  # Handles only text features

        self.dropout = torch.nn.Dropout(p=0.1)
        self.activation = torch.nn.ReLU()
    def forward(self, text_features, image_features=None):
        text_out = self.dropout(self.activation(self.text_processor(text_features)))
        if image_features is not None:
             image_out = self.dropout(self.activation(self.image_processor(image_features)))
             combined_features = torch.cat((text_out, image_out), dim=1)
             output = torch.log_softmax(self.combined_processor(combined_features), dim=1)
        else:
             output = torch.log_softmax(self.text_only_processor(text_out), dim=1)  # Use the text-only processor
        return output
    def cross_entropy_loss(self, logits, labels):
        return F.nll_loss(logits, labels)
    
    def training_step(self, train_batch, batch_idx):
        lab,txt= train_batch
        lab = train_batch[lab]
        txt = train_batch[txt]
        logit_offen= self.forward(txt)
        loss = self.cross_entropy_loss(logit_offen, lab)
        self.log('train_loss', loss)
        return loss
    def validation_step(self, val_batch, batch_idx):
        lab,txt,= val_batch
        lab = val_batch[lab]
        #print(lab)
          #print(lab)
 
        txt = val_batch[txt]
        
        #   img = val_batch[img]
        logits= self.forward(txt)
        tmp = np.argmax(logits.detach().cpu().numpy(),axis=-1)
        loss = self.cross_entropy_loss(logits, lab)
        lab = lab.detach().cpu().numpy()
        self.log('val_acc', accuracy_score(lab,tmp))
        self.log('val_roc_auc',roc_auc_score(lab,tmp))
        self.log('val_loss', loss)
        tqdm_dict = {'val_acc': accuracy_score(lab,tmp)}
        return {
                'progress_bar': tqdm_dict,
      'val_f1 offensive': f1_score(lab,tmp,average='macro')
      }
    def validation_epoch_end(self, validation_step_outputs):
        outs = []
        outs1,outs2,outs3,outs4,outs5,outs6,outs7,outs8,outs9,outs10,outs11,outs12,outs13,outs14,outs16,outs17 = \
    [],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
        for out in validation_step_outputs:
            outs.append(out['progress_bar']['val_acc'])
            outs14.append(out['val_f1 offensive'])
        self.log('val_acc_all_offn', sum(outs)/len(outs))
        self.log('val_f1 offensive', sum(outs14)/len(outs14))
        print(f'***val_acc_all_offn at epoch end {sum(outs)/len(outs)}****')
        print(f'***val_f1 offensive at epoch end {sum(outs14)/len(outs14)}****')
    def test_step(self, batch, batch_idx):
        lab,txt= batch

        lab = batch[lab]
      #print(lab)
        txt = batch[txt]
    #   img = batch[img]
        logits= self.forward(txt)
        tmp = np.argmax(logits.detach().cpu().numpy(),axis=-1)
        loss = self.cross_entropy_loss(logits, lab)
        lab = lab.detach().cpu().numpy()
        self.log('test_acc', accuracy_score(lab,tmp))
        self.log('test_roc_auc',roc_auc_score(lab,tmp))
        self.log('test_loss', loss)
        tqdm_dict = {'test_acc': accuracy_score(lab,tmp)}
        return {
                'progress_bar': tqdm_dict,
                'test_acc': accuracy_score(lab,tmp),
                'test_f1_score': f1_score(lab,tmp,average='macro'),}
    def test_epoch_end(self, outputs):
        outs = []
        outs1,outs2,outs3,outs4,outs5,outs6,outs7,outs8,outs9,outs10,outs11,outs12,outs13,outs14 = \
      [],[],[],[],[],[],[],[],[],[],[],[],[],[]
        for out in outputs:
            outs.append(out['test_acc'])
            outs2.append(out['test_f1_score'])
        self.log('test_acc', sum(outs)/len(outs))
        self.log('test_f1_score', sum(outs2)/len(outs2))
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=3e-5)
        return optimizer


class HmDataModule(pl.LightningDataModule):
    def setup(self, stage):  
        self.hm_train = t_p
        self.hm_val = v_p
    # self.hm_test = test
        self.hm_test = te_p
    def train_dataloader(self):
        return DataLoader(self.hm_train, batch_size=128, drop_last=True)
    def val_dataloader(self):
        return DataLoader(self.hm_val, batch_size=128, drop_last=True)
    def test_dataloader(self):
        return DataLoader(self.hm_test, batch_size=128, drop_last=True)

data_module = HmDataModule()
checkpoint_callback = ModelCheckpoint(
     monitor='val_acc_all_offn',
     dirpath='dehate/',
     filename='epoch{epoch:02d}-val_f1_all_offn{val_acc_all_offn:.2f}',
     auto_insert_metric_name=False,
     save_top_k=1,
    mode="max",
 )
all_callbacks = []
all_callbacks.append(checkpoint_callback)
# train
from pytorch_lightning import seed_everything
seed_everything(42, workers=True)
hm_model = Classifier()
gpus = 1 if torch.cuda.is_available() else 0
trainer = pl.Trainer(gpus=gpus,deterministic=True,max_epochs=60,precision=16,callbacks=all_callbacks)
# trainer = pl.Trainer(gpus=gpus,max_epochs=10,callbacks=all_callbacks)
trainer.fit(hm_model, data_module)


Global seed set to 42
/DATA/atul_2221cs20/anaconda3/envs/jitu/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:479: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/DATA/atul_2221cs20/anaconda3/envs/jitu/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory /DATA/atul_2221cs20/jitendra/New_model/dehate exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]

  | Name                | Type    | Params
-------

Sanity Checking: 0it [00:00, ?it/s]

***val_acc_all_offn at epoch end 0.4296875****
***val_f1 offensive at epoch end 0.4145299145299145****


/DATA/atul_2221cs20/anaconda3/envs/jitu/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:229: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/DATA/atul_2221cs20/anaconda3/envs/jitu/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:229: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

***val_acc_all_offn at epoch end 0.6506696428571429****
***val_f1 offensive at epoch end 0.6471136763066608****


Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=60` reached.


***val_acc_all_offn at epoch end 0.6529017857142857****
***val_f1 offensive at epoch end 0.6492255991468674****


In [19]:
test_dataloader = DataLoader(dataset=te_p, batch_size=1478)
ckpt_path = '/DATA/atul_2221cs20/jitendra/New_model/dehate/epoch27-val_f1_all_offn0.67.ckpt' # put ckpt_path according to the path output in the previous cell
trainer.test(dataloaders=test_dataloader,ckpt_path=ckpt_path)

Restoring states from the checkpoint path at /DATA/atul_2221cs20/jitendra/New_model/dehate/epoch27-val_f1_all_offn0.67.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]
Loaded model weights from checkpoint at /DATA/atul_2221cs20/jitendra/New_model/dehate/epoch27-val_f1_all_offn0.67.ckpt
/DATA/atul_2221cs20/anaconda3/envs/jitu/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:229: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.6611601598394191     │
│       test_f1_score       │    0.6598244038813443     │
│         test_loss         │    0.6121626496315002     │
│       test_roc_auc        │    0.6605892491981492     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.6611601598394191,
  'test_roc_auc': 0.6605892491981492,
  'test_loss': 0.6121626496315002,
  'test_f1_score': 0.6598244038813443}]

In [21]:
model_path = '/DATA/atul_2221cs20/jitendra/New_model/dehate/epoch27-val_f1_all_offn0.67.ckpt'
trained_model = Classifier.load_from_checkpoint(checkpoint_path=model_path)


In [22]:
print(trained_model)

Classifier(
  (text_processor): Linear(in_features=768, out_features=256, bias=True)
  (image_processor): Linear(in_features=512, out_features=256, bias=True)
  (combined_processor): Linear(in_features=512, out_features=2, bias=True)
  (text_only_processor): Linear(in_features=256, out_features=2, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (activation): ReLU()
)


In [23]:
#for testing

In [45]:
data_test = pd.read_csv('/DATA/atul_2221cs20/jitendra/MAMI_test_set.csv') 
def get_data1(data_test):  # data = pd.read_csv(dataset_path)
    # Extracting lists from the DataFrame
    text = list(data_test['Text_Transcription'])
    img_path = list(data_test['file_name'])
    name = list(data_test['file_name'])
    label = list(data_test['misogynous'])

    # Initializing lists for storing processed data
    text_features, image_features, l, Name, v, text_descriptions = [], [], [], [], [], []

    for txt, img, L, n in tqdm(zip(text, img_path, label, name)):
        try:
            img = Image.open('/DATA/atul_2221cs20/jitendra/Jitendra/data/test_images/' + img)
        except Exception as e:
            print(e)
            continue

        img = torch.stack([compose(img).to(device)])

        l.append(L)
        Name.append(n)
        text_descriptions.append(txt)  # Append the text description to the list
      
        with torch.no_grad():
           # txt = clip.tokenize(txt,truncate=True).to(device)
            temp_txt = model_t.forward(txt, tokenizer).detach().cpu().numpy()
            #print(temp_txt.shape)
#             temp_txt = model.forward(txt, tokenizer).detach().cpu().numpy()
            
            text_features.append(temp_txt)
            

            temp_img = clip_model.encode_image(img).detach().cpu().numpy()
            #temp_img = model.forward(img).detach().cpu().numpy()
           # print(temp_img.shape)
            image_features.append(temp_img)

            del temp_txt
            del temp_img
            torch.cuda.empty_cache()

        del img
        torch.cuda.empty_cache()

    # Return the text descriptions along with the other data
    return text_features, image_features, l, Name, text_descriptions
class HatefulDataset1(Dataset):
    def __init__(self, data):
        # Assuming get_data now also returns text_descriptions
        self.t_f, self.i_f, self.label, self.name, self.text_descriptions = get_data1(data)
        
        # Squeeze the arrays if necessary (assuming this is for removing unnecessary dimensions)
        self.t_f = np.squeeze(np.asarray(self.t_f), axis=1)
        self.i_f = np.squeeze(np.asarray(self.i_f), axis=1)
    
    def __len__(self):
        # The length of the dataset is the length of the labels
        return len(self.label)
    
    def __getitem__(self, idx):
        # Convert idx from tensor to list if it's a tensor
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        # Extract data for the given index
        name = self.name[idx]
        label = self.label[idx]
        T = self.t_f[idx, :]
        I = self.i_f[idx, :]
        text_description = self.text_descriptions[idx]  # Extract the text description
        
        # Create a sample dictionary to return
        sample = {
            'label': label,
            'processed_txt': T,
            'processed_img': I,
            'name': name,
            'text_description': text_description  # Include the text description in the sample
        }
        
        return sample
    

In [46]:
sample_dataset_t = HatefulDataset1(data_test)

0it [00:00, ?it/s]

In [47]:
torch.save(sample_dataset_t,'MAMI_test3.pt')

In [48]:
sample_dataset_test= torch.load("MAMI_test3.pt")

In [49]:
sample_dataset_test[1]

{'label': 0,
 'processed_txt': array([ 3.16702753e-01, -2.64505565e-01,  2.90357322e-01,  4.40030038e-01,
         2.41311908e-01,  3.03467631e-01, -1.85672283e-01,  4.47987050e-01,
        -3.47782448e-02, -2.17394665e-01, -2.53920276e-02, -1.29242882e-01,
         4.14236695e-01,  1.74275324e-01,  3.52811068e-01, -2.43612587e-01,
         3.03050399e-01,  4.99805361e-01, -7.39374831e-02, -6.65779769e-01,
        -1.81546316e-01, -3.48220468e-01,  2.33768255e-01,  1.41589716e-01,
         2.53372937e-01, -4.33969021e-01,  1.11369193e-01, -2.38649696e-01,
         3.41512188e-02,  3.14746231e-01,  2.47805238e-01, -4.91967916e-01,
         9.64541081e-03,  2.59685308e-01, -2.23373950e-01, -3.97437364e-02,
         5.96575066e-02,  9.93316714e-03, -1.91678241e-01, -1.48252904e-01,
         3.10360994e-02,  4.73225594e-01,  1.19926579e-01, -4.99491021e-02,
         5.68662770e-02, -2.03625977e-01, -2.04187855e-01, -1.67487115e-01,
         3.27440023e-01, -4.24950898e-01,  1.21770784e-01,

In [51]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import pytorch_lightning as pl

class Classifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.text_processor = torch.nn.Linear(768, 256)
        self.image_processor = torch.nn.Linear(512, 256)
        self.combined_processor = torch.nn.Linear(512, 2)
        self.text_only_processor = torch.nn.Linear(256, 2)
        self.dropout = torch.nn.Dropout(0.1)
        self.activation = torch.nn.ReLU()

    def forward(self, text_features, image_features=None):
        text_out = self.dropout(self.activation(self.text_processor(text_features)))
        if image_features is not None:
            image_out = self.dropout(self.activation(self.image_processor(image_features)))
            combined_features = torch.cat((text_out, image_out), dim=1)
            return torch.log_softmax(self.combined_processor(combined_features), dim=1)
        else:
            return torch.log_softmax(self.text_only_processor(text_out), dim=1)

def load_model(checkpoint_path):
    model = Classifier.load_from_checkpoint(checkpoint_path)
    model.eval()
    model.freeze()
    return model

class HmDataModule(pl.LightningDataModule):
    def __init__(self, test_dataset):
        super().__init__()
        self.test_dataset = test_dataset

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=128, shuffle=False)

def run_inference(model, data_module):
    class_counts = [0, 0]  # Assuming two classes
    dataloader = data_module.test_dataloader()

    with torch.no_grad():
        for batch in dataloader:
            text_features, image_features = batch['processed_txt'].float(), batch['processed_img'].float()
            outputs = model(text_features, image_features)
            predictions = torch.argmax(outputs, dim=1)
            for pred in predictions:
                class_counts[pred.item()] += 1

    total = sum(class_counts)
    percentages = [count / total * 100 for count in class_counts]
    return percentages

# Setup and usage
checkpoint_path = '/DATA/atul_2221cs20/jitendra/New_model/dehate/epoch27-val_f1_all_offn0.67-v1.ckpt'
model = load_model(checkpoint_path)
test_dataset = torch.load("MAMI_test3.pt")
data_module = HmDataModule(test_dataset=test_dataset)

# Run inference
percentages = run_inference(model, data_module)
print("Class distribution:", percentages)
print("ethic:"+str(percentages[0])+"%")
print("non-ethic:"+str(percentages[1])+"%")


Class distribution: [49.2, 50.8]
ethic:49.2%
non-ethic:50.8%


In [47]:
#now further we will  analysis our newly created dataset

In [37]:
data_test1 = pd.read_csv('/DATA/atul_2221cs20/jitendra/New_model/data/result.csv') 
def get_data2(data_test):  # data = pd.read_csv(dataset_path)
    # Extracting lists from the DataFrame
    text = list(data_test['text'])
    img_path = list(data_test['name'])
    name = list(data_test['name'])
    label = list(data_test['label'])

    # Initializing lists for storing processed data
    text_features, image_features, l, Name, v, text_descriptions = [], [], [], [], [], []

    for txt, img, L, n in tqdm(zip(text, img_path, label, name)):
        try:
            img = Image.open('/DATA/atul_2221cs20/jitendra/New_model2/output_images_label_02/' + img)
        except Exception as e:
            print(e)
            continue

        img = torch.stack([compose(img).to(device)])

        l.append(L)
        Name.append(n)
        text_descriptions.append(txt)  # Append the text description to the list
        #print(txt)
        with torch.no_grad():
           # txt = clip.tokenize(txt,truncate=True).to(device)
            temp_txt = model_t.forward(txt, tokenizer).detach().cpu().numpy()
            #print(temp_txt.shape)
#             temp_txt = model.forward(txt, tokenizer).detach().cpu().numpy()
            
            text_features.append(temp_txt)
            

            temp_img = clip_model.encode_image(img).detach().cpu().numpy()
            #temp_img = model.forward(img).detach().cpu().numpy()
           # print(temp_img.shape)
            image_features.append(temp_img)

            del temp_txt
            del temp_img
            torch.cuda.empty_cache()

        del img
        torch.cuda.empty_cache()

    # Return the text descriptions along with the other data
    return text_features, image_features, l, Name, text_descriptions
class HatefulDataset2(Dataset):
    def __init__(self, data):
        # Assuming get_data now also returns text_descriptions
        self.t_f, self.i_f, self.label, self.name, self.text_descriptions = get_data2(data)
        
        # Squeeze the arrays if necessary (assuming this is for removing unnecessary dimensions)
        self.t_f = np.squeeze(np.asarray(self.t_f), axis=1)
        self.i_f = np.squeeze(np.asarray(self.i_f), axis=1)
    
    def __len__(self):
        # The length of the dataset is the length of the labels
        return len(self.label)
    
    def __getitem__(self, idx):
        # Convert idx from tensor to list if it's a tensor
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        # Extract data for the given index
        name = self.name[idx]
        label = self.label[idx]
        T = self.t_f[idx, :]
        I = self.i_f[idx, :]
        text_description = self.text_descriptions[idx]  # Extract the text description
        
        # Create a sample dictionary to return
        sample = {
            'label': label,
            'processed_txt': T,
            'processed_img': I,
            'name': name,
            'text_description': text_description  # Include the text description in the sample
        }
        
        return sample
    

In [38]:
sample_dataset_tune = HatefulDataset2(data_test1)

0it [00:00, ?it/s]

In [39]:
torch.save(sample_dataset_tune,'MAMI_tune5.pt')

In [40]:
sample_dataset_test= torch.load("MAMI_tune5.pt")

In [41]:
sample_dataset_test[5]

{'label': 1,
 'processed_txt': array([ 2.82500088e-01,  1.83196336e-01, -1.67982221e-01,  1.50857374e-01,
        -4.27507013e-02, -3.59965116e-02,  6.80844411e-02, -1.14675738e-01,
        -1.01015195e-01, -1.86859928e-02,  4.85697128e-02, -5.86252958e-02,
         5.95894754e-02,  4.20276791e-01, -5.42770885e-02, -2.03694031e-03,
         2.90705502e-01,  1.22283399e-02,  7.34806731e-02, -2.00048499e-02,
         1.01662531e-01, -9.98960882e-02,  3.59894708e-02, -2.31263954e-02,
        -3.54237221e-02,  2.42099762e-01,  7.71585032e-02,  3.99403960e-01,
        -4.51284975e-01, -7.06431866e-02, -2.85889745e-01, -2.30849087e-01,
        -1.06287979e-01,  2.83340774e-02,  3.58357936e-01,  1.27430961e-01,
         2.00547859e-01,  6.66579604e-02,  1.25808761e-01, -2.14498729e-01,
         5.50239794e-02, -2.56351113e-01,  4.55249585e-02, -2.87044980e-02,
         1.35561183e-01, -2.03634799e-01,  1.65566295e-01,  1.88449755e-01,
         1.47859856e-01, -1.96928486e-01,  1.68458313e-01,

In [52]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import pytorch_lightning as pl

class Classifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.text_processor = torch.nn.Linear(768, 256)
        self.image_processor = torch.nn.Linear(512, 256)
        self.combined_processor = torch.nn.Linear(512, 2)
        self.text_only_processor = torch.nn.Linear(256, 2)
        self.dropout = torch.nn.Dropout(0.1)
        self.activation = torch.nn.ReLU()

    def forward(self, text_features, image_features=None):
        text_out = self.dropout(self.activation(self.text_processor(text_features)))
        if image_features is not None:
            image_out = self.dropout(self.activation(self.image_processor(image_features)))
            combined_features = torch.cat((text_out, image_out), dim=1)
            return torch.log_softmax(self.combined_processor(combined_features), dim=1)
        else:
            return torch.log_softmax(self.text_only_processor(text_out), dim=1)

def load_model(checkpoint_path):
    modelc = Classifier.load_from_checkpoint(checkpoint_path)
    modelc.eval()
    modelc.freeze()
    return modelc

class HmDataModule(pl.LightningDataModule):
    def __init__(self, test_dataset):
        super().__init__()
        self.test_dataset = test_dataset

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=128, shuffle=False)

def run_inference(modelc, data_module):
    class_counts = [0, 0]  # Assuming two classes
    
    dataloader = data_module.test_dataloader()

    with torch.no_grad():
        for batch in dataloader:
            text_features, image_features = batch['processed_txt'].float(), batch['processed_img'].float()
            outputs = modelc(text_features, image_features)
            predictions = torch.argmax(outputs, dim=1)
            for pred in predictions:
                class_counts[pred.item()] += 1

    total = sum(class_counts)
    percentages = [count / total * 100 for count in class_counts]
    return percentages

#Setup and usage
checkpoint_path = '/DATA/atul_2221cs20/jitendra/New_model/dehate/epoch27-val_f1_all_offn0.67-v1.ckpt'
modelc = load_model(checkpoint_path)
test_dataset = torch.load("MAMI_tune5.pt")
data_module = HmDataModule(test_dataset=test_dataset)

# Run inference
percentages = run_inference(modelc, data_module)
print("ethic:"+str(percentages[0])+"%")
print("non-ethic:"+str(percentages[1])+"%")




ethic:59.099999999999994%
non-ethic:40.9%
